In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
!pip install MulticoreTSNE

%matplotlib inline
from MulticoreTSNE import MulticoreTSNE as TSNE
from matplotlib import pyplot as plt
import torch
import copy
from torchvision import datasets, transforms
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd 
from sklearn.preprocessing import Normalizer
from tqdm import tqdm_notebook
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
UNLABELED_BS = 256
TRAIN_BS = 32
TEST_BS = 1024

total_samples = 60000
label_unlabel_ratio = 0.1
num_train_samples = (int)(label_unlabel_ratio * total_samples)

x = pd.read_csv('/content/drive/MyDrive/mnistDataset/mnist_train.csv')
y = x['label']
x.drop(['label'], inplace = True, axis = 1)

x_test = pd.read_csv('/content/drive/MyDrive/mnistDataset/mnist_test.csv')
y_test = x_test['label']
x_test.drop(['label'], inplace = True, axis = 1)

normalizer = Normalizer()

In [ ]:
# Architecture from : https://github.com/peimengsui/semi_supervised_mnist
class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
            self.conv2 = nn.Conv2d(20, 40, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(640, 150)
            self.fc2 = nn.Linear(150, 10)
            self.log_softmax = nn.LogSoftmax(dim = 1)

        def forward(self, x):
            x = x.view(-1,1,28,28)
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 640)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = F.relu(self.fc2(x))
            x = self.log_softmax(x)
            return x
        
net = Net().cuda()

In [ ]:
def getClientDataset(num_clients, x, y):

    
    
    x_lab = x.iloc[0:num_train_samples]
    y_lab = y.iloc[0:num_train_samples]
    
    x_unlab = x.iloc[num_train_samples:]

    samples_per_client_lab = (int)(x_lab.shape[0]/num_clients)
    samples_per_client_unlab = (int)(x_unlab.shape[0]/num_clients)

    client_train_lab = []
    client_train_unlab = []

    for i in range(num_clients):

        xlab = x_lab.iloc[i*samples_per_client_lab:(i+1)*samples_per_client_lab, :].values
        ylab = y_lab.iloc[i*samples_per_client_lab:(i+1)*samples_per_client_lab].values
        xunlab = x_unlab.iloc[i*samples_per_client_unlab:(i+1)*samples_per_client_unlab, :].values
        print(xlab.shape)
        print(ylab.shape)
        print(xunlab.shape)
        xlab = normalizer.fit_transform(xlab)
        xunlab = normalizer.transform(xunlab)

        xlab = torch.from_numpy(xlab).type(torch.FloatTensor)
        ylab = torch.from_numpy(ylab).type(torch.LongTensor) 

        train = torch.utils.data.TensorDataset(xlab, ylab)

        train_loader = torch.utils.data.DataLoader(train, batch_size = TRAIN_BS, shuffle = True, num_workers = 8)

        unlabeled_train = torch.from_numpy(xunlab).type(torch.FloatTensor)

        unlabeled = torch.utils.data.TensorDataset(unlabeled_train)
        unlabeled_loader = torch.utils.data.DataLoader(unlabeled, batch_size = UNLABELED_BS, shuffle = True, num_workers = 8)

        client_train_lab.append(train_loader)
        client_train_unlab.append(unlabeled_loader)

    return client_train_lab, client_train_unlab


def getTestData(x_test, y_test):

    x_test = normalizer.transform(x_test.values)

    x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
    y_test = torch.from_numpy(y_test.values).type(torch.LongTensor)

    test = torch.utils.data.TensorDataset(x_test, y_test)

    test_loader = torch.utils.data.DataLoader(test, batch_size = TEST_BS, shuffle = True, num_workers = 8)

    return test_loader


def getClientModels(num_clients):

    client_models = []

    for i in range(num_clients):
        client_models.append(Net().cuda())
      
    return client_models


def getGlobalModel():

    return Net().cuda()


def updateGlobalModel(num_client, client_models, global_model):

    with torch.no_grad():

        conv1_wts = client_models[0].conv1.weight.data    
        conv2_wts = client_models[0].conv2.weight.data
        fc1_wts = client_models[0].fc1.weight.data
        fc2_wts = client_models[0].fc2.weight.data

        conv1_b = client_models[0].conv1.bias.data    
        conv2_b = client_models[0].conv2.bias.data
        fc1_b = client_models[0].fc1.bias.data
        fc2_b = client_models[0].fc2.bias.data


        for i in range(1,num_clients):

            conv1_wts += client_models[i].conv1.weight.data    
            conv2_wts += client_models[i].conv2.weight.data
            fc1_wts += client_models[i].fc1.weight.data
            fc2_wts += client_models[i].fc2.weight.data

            conv1_b += client_models[i].conv1.bias.data    
            conv2_b += client_models[i].conv2.bias.data
            fc1_b += client_models[i].fc1.bias.data
            fc2_b += client_models[i].fc2.bias.data

        global_model.conv1.weight.set_(conv1_wts/num_clients)
        global_model.conv2.weight.set_(conv2_wts/num_clients)
        global_model.fc1.weight.set_(fc1_wts/num_clients)
        global_model.fc2.weight.set_(fc2_wts/num_clients)

        global_model.conv1.bias.set_(conv1_b/num_clients)
        global_model.conv2.bias.set_(conv2_b/num_clients)
        global_model.fc1.bias.set_(fc1_b/num_clients)
        global_model.fc2.bias.set_(fc2_b/num_clients)


    return global_model





In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0 
    loss = 0
    i=0
    with torch.no_grad():
        for data, labels in test_loader:
            
            data = data.cuda()
            output = model(data)
            predicted = torch.max(output,1)[1]
            correct += (predicted == labels.cuda()).sum()
            loss += F.nll_loss(output, labels.cuda()).item()
            i+=predicted.shape[0]
    return (float(correct)/i) *100, (loss/len(test_loader))

In [ ]:
def train_supervised(model, train_loader, test_loader, EPOCHS):
    optimizer = torch.optim.SGD( model.parameters(), lr = 0.1)
    model.train()

    for epoch in tqdm_notebook(range(EPOCHS)):
        correct = 0
        running_loss = 0
        i=0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            i+=y_batch.shape[0]
            output = model(X_batch)
            labeled_loss = F.nll_loss(output, y_batch)
                       
            optimizer.zero_grad()
            labeled_loss.backward()
            optimizer.step()
            running_loss += labeled_loss.item()
        
        if epoch %1 == 0:
            test_acc, test_loss = evaluate(model, test_loader)
            print('Epoch: {} : Train Loss : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, running_loss/(1 * i), test_acc, test_loss))
            model.train()

In [ ]:
T1 = 100
T2 = 700
af = 3

def alpha_weight(epoch):
    if epoch < T1:
        return 0.0
    elif epoch > T2:
        return af
    else:
         return ((epoch-T1) / (T2-T1))*af

In [ ]:
from tqdm import tqdm_notebook

acc_scores = []
unlabel = []
pseudo_label = []

alpha_log = []
test_acc_log = []
test_loss_log = []
def semisup_train(model, train_loader, unlabeled_loader, test_loader, EPOCHS):
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
    
    # Instead of using current epoch we use a "step" variable to calculate alpha_weight
    # This helps the model converge faster
    step = 100 
    
    model.train()
    for epoch in tqdm_notebook(range(EPOCHS)):
        for batch_idx, x_unlabeled in enumerate(unlabeled_loader):
            
            
            # Forward Pass to get the pseudo labels
            x_unlabeled = x_unlabeled[0].cuda()
            model.eval()
            output_unlabeled = model(x_unlabeled)
            _, pseudo_labeled = torch.max(output_unlabeled, 1)
            model.train()

                        
            
            """ ONLY FOR VISUALIZATION"""
            if (batch_idx < 3) and (epoch % 10 == 0):
                unlabel.append(x_unlabeled.cpu())
                pseudo_label.append(pseudo_labeled.cpu())
            """ ********************** """
            
            # Now calculate the unlabeled loss using the pseudo label
            output = model(x_unlabeled)
            unlabeled_loss = alpha_weight(step) * F.nll_loss(output, pseudo_labeled)   
            
            # Backpropogate
            optimizer.zero_grad()
            unlabeled_loss.backward()
            optimizer.step()
            
            
            # For every 50 batches train one epoch on labeled data 
            if batch_idx % 50 == 0:
                
                # Normal training procedure
                for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
                    X_batch = X_batch.cuda()
                    y_batch = y_batch.cuda()
                    output = model(X_batch)
                    labeled_loss = F.nll_loss(output, y_batch)

                    optimizer.zero_grad()
                    labeled_loss.backward()
                    optimizer.step()

                                    
                # Now we increment step by 1
                step += 1
                

        test_acc, test_loss =evaluate(model, test_loader)
        print('Epoch: {} : Alpha Weight : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, alpha_weight(step), test_acc, test_loss))
        
        """ LOGGING VALUES """
        alpha_log.append(alpha_weight(step))
        test_acc_log.append(test_acc/100)
        test_loss_log.append(test_loss)
        """ ************** """
        model.train()

In [ ]:
num_global_iter = 5
num_clients = 4
num_local_epochs = 5

print("======================================== !!! Starting Federated Learning !!! ==================================================")

client_models = getClientModels(num_clients)
global_model = getGlobalModel()

client_train_lab, client_train_unlab = getClientDataset(num_clients, x, y)
test_loader = getTestData(x_test, y_test)

for iter in range(num_global_iter):

    print("============================================ Global Training Iteration : " + str(iter+1) + " ========================================\n\n")

    for i in range(num_clients):
        
        print("==================================== Supervised Training on Client : " + str(i+1) + "============================================\n")

        train_supervised(client_models[i], client_train_lab[i], test_loader, num_local_epochs)

        print("====================================== Semi Supervised Alpha Weighted Training =========================================\n")

        semisup_train(client_models[i], client_train_lab[i], client_train_unlab[i], test_loader, num_local_epochs)

    global_model = updateGlobalModel(num_clients, client_models, global_model)

    test_acc, test_loss = evaluate(global_model, test_loader)

    print("Global Model Accuracy : ", test_acc)
    print("Global Model Loss : ", test_loss)

    client_models = []

    for i in range(num_clients):
        client_models.append(copy.deepcopy(global_model))


======================================== !!! Starting Federated Learning !!! ==================================================
(1500, 784)
(1500,)
(13500, 784)
(1500, 784)
(1500,)
(13500, 784)
(1500, 784)
(1500,)
(13500, 784)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: 

(1500, 784)
(1500,)
(13500, 784)
============================================ Global Training Iteration : 1 ========================================


==================================== Supervised Training on Client : 1============================================



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: 

  0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch: 0 : Alpha Weight : 0.01000 | Test Acc : 95.86000 | Test Loss : 0.135 
Epoch: 1 : Alpha Weight : 0.02000 | Test Acc : 95.49000 | Test Loss : 0.150 
Epoch: 2 : Alpha Weight : 0.03000 | Test Acc : 95.76000 | Test Loss : 0.144 
Epoch: 3 : Alpha Weight : 0.04000 | Test Acc : 95.81000 | Test Loss : 0.140 
Epoch: 4 : Alpha Weight : 0.05000 | Test Acc : 95.68000 | Test Loss : 0.153 
==================================== Supervised Training on Client : 4============================================



  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 : Train Loss : 0.00493 | Test Acc : 96.13000 | Test Loss : 0.127 
Epoch: 1 : Train Loss : 0.00539 | Test Acc : 95.91000 | Test Loss : 0.131 
Epoch: 2 : Train Loss : 0.00502 | Test Acc : 95.70000 | Test Loss : 0.142 
Epoch: 3 : Train Loss : 0.00466 | Test Acc : 95.95000 | Test Loss : 0.122 
Epoch: 4 : Train Loss : 0.00452 | Test Acc : 95.90000 | Test Loss : 0.131 
====================================== Semi Supervised Alpha Weighted Training =========================================



  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 : Alpha Weight : 0.01000 | Test Acc : 95.93000 | Test Loss : 0.139 
Epoch: 1 : Alpha Weight : 0.02000 | Test Acc : 95.76000 | Test Loss : 0.141 
Epoch: 2 : Alpha Weight : 0.03000 | Test Acc : 95.85000 | Test Loss : 0.140 
Epoch: 3 : Alpha Weight : 0.04000 | Test Acc : 95.73000 | Test Loss : 0.141 
Epoch: 4 : Alpha Weight : 0.05000 | Test Acc : 95.85000 | Test Loss : 0.143 
Global Model Accuracy :  96.46000000000001
Global Model Loss :  0.11153065487742424


In [ ]:
test_acc, test_loss = evaluate(global_model, test_loader)
print('Test Acc : {:.5f} | Test Loss : {:.3f} '.format(test_acc, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test Acc : 96.46000 | Test Loss : 0.112 
